<a href="https://colab.research.google.com/github/illliyas/Machine-Learning/blob/master/Topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import matplotlib.pyplot as plt
from math import log, sqrt
import pandas as pd
import numpy as np
import re
import spacy
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-t

In [ ]:
!python -m spacy download en_core_web_sm
!pip install locationtagger

2022-10-23 19:25:05.951024: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 2.6 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import locationtagger

In [ ]:
df = pd.read_csv('ek_tweets2.csv')

In [ ]:
def cleaning_tweets(tweet):
    temp = tweet.lower()
    #Removing hashtags and mentions
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    #Removing links
    temp = re.sub(r"http\S+", "", temp)
    temp = re.sub(r"www.\S+", "", temp)
    #Removing punctuations
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    #Filtering non-alphanumeric characters
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = re.sub("([a-z]*[0-9]+[a-z0-9]* )+"," ", temp)
    #remove 2 letter words
    temp = re.sub(r"\b[a-z]{2}\b"," ", temp)
    return temp
sw = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
def remove_stopwords(tweet):
    words = word_tokenize(tweet)
    words = [word for word in words if word not in sw]
    words = [lemmatizer.lemmatize(word) for word in words]
    #words = [stemmer.stem(word) for word in words]
    return ' '.join(words) 
def get_places(tweet):
    place_entity = locationtagger.find_locations(text = tweet)
    places_list = [y for x in [ place_entity.countries + place_entity.regions + place_entity.cities] for y in x]
    places_list = [x.lower() for x in places_list]
    places_list = list(dict.fromkeys(places_list))
    return ' '.join(places_list)
def remove_places(tweet):
    place_entity = locationtagger.find_locations(text = tweet)
    places_list = [y for x in [ place_entity.countries + place_entity.regions + place_entity.cities] for y in x]
    places_list = [x.lower() for x in places_list]
    places_list = list(dict.fromkeys(places_list))
    words = word_tokenize(tweet)
    words = [i for i in words if i not in places_list]
    return ' '.join(words)
def number_of_words(tweet):
  return len(word_tokenize(tweet))    

In [ ]:
df['tweet_clean'] = df.apply(lambda row: cleaning_tweets(row['tweet']),axis=1)
df['tweet_clean2'] = df.apply(lambda row: remove_stopwords(row['tweet_clean']),axis=1)
df['tweet_places'] = df.apply(lambda row: get_places(row['tweet_clean2']),axis=1)
df['tweet_clean3'] = df.apply(lambda row: remove_places(row['tweet_clean2']),axis=1)
df['number_of_words'] = df.apply(lambda row: number_of_words(row['tweet_clean3']),axis=1)

In [ ]:
list(df['tweet'])[:5]

['@EmiratesSupport Thank you for your quick response. I have sent the required information.',
 'RT @smartykapu: @EmiratesSupport We have already Send our formal complaint.. Waiting for your revert..',
 '@EmiratesSupport So being silver member where I have complimentary seat selection, I am restricted to select seats? https://t.co/FLVE2Xu3uY',
 '@EmiratesSupport @emirates have attempted booking a flight online 3x- all transactions showing up as "failed" on the website however it has been charged to the account? Please help',
 '@EmiratesSupport did thanks']

In [ ]:
list(df['tweet_clean3'])[:5]

['thank quick response sent required information',
 'already send formal complaint waiting revert',
 'silver member complimentary seat selection restricted select seat',
 'attempted booking flight online transaction showing failed website however charged account please help',
 'thanks']

In [ ]:
df2 = df[df['number_of_words']>2]

In [ ]:
df2=df2.reset_index()

In [ ]:
df2.to_csv('ek_tweets_clean.csv',index=False)

In [ ]:
---------------------

In [ ]:
!pip install --upgrade joblib==1.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install joblib==1.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 306 kB 5.3 MB/s 
  Attempting uninstall: joblib
    Found existing installation: joblib 1.2.0
    Uninstalling joblib-1.2.0:
      Successfully uninstalled joblib-1.2.0


In [ ]:
from bertopic import BERTopic

In [ ]:
from sklearn.datasets import fetch_20newsgroups
 
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

In [ ]:
tweets = df2.tweet_clean2.to_list()
timestamp  = df2.created_at.to_list()

In [ ]:
model = BERTopic(verbose=True,embedding_model='paraphrase-MiniLM-L3-v2', min_topic_size= 3)
topics, _ = model.fit_transform(tweets)

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

2022-10-23 20:02:46,090 - BERTopic - Transformed documents to Embeddings
2022-10-23 20:02:50,830 - BERTopic - Reduced dimensionality
2022-10-23 20:02:50,869 - BERTopic - Clustered reduced embeddings


In [ ]:
model = BERTopic(verbose=True,embedding_model='all-mpnet-base-v2', min_topic_size= 3)
topics, _ = model.fit_transform(tweets)

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

2022-10-23 19:52:47,568 - BERTopic - Transformed documents to Embeddings
2022-10-23 19:52:51,252 - BERTopic - Reduced dimensionality
2022-10-23 19:52:51,292 - BERTopic - Clustered reduced embeddings


In [ ]:
model.get_topic_info()

,Topic,Count,Name
0,-1,65,-1_flight_received_emirate_get
1,0,31,0_booking_change_contact_help
2,1,31,1_complaint_customer_feedback_response
3,2,27,2_dubai_airport_visa_hour
4,3,13,3_saturday_system_decided_shoul
5,4,13,4_much_thanks_resolving_appreciated
6,5,13,5_nut_food_offer_cater
7,6,12,6_refund_rebook_flight_cancelled
8,7,11,7_baggage_bag_luggage_missing
9,8,10,8_luggage_extra_carry_per


In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_barchart()